## This notebook shows how to use get FABRIC tokens for Jupyterhub users

In [4]:
import os
from fabric.cli.credential import CredMgr
from fabric.cli.orchestrator import Orchestrator
import json

### Fabric Tokens
Fabric has 2 kinds of tokens:<br>
- Identity : required for Control/Measurement Framework APIs. Identity Token is valid upto an hour.
- Refresh : required to generate new Identity Tokens valid. Refresh Token is valid for 24 hours.

Fabric Identity token is required for Control/Measurement Framework APIs.<br>

When user logins to Jupyterhub after authenticating against CILogon, OIDC refresh token is derived. <br>
This token is available as the environment variable `CILOGON_REFRESH_TOKEN`.<br>

On the first login, we use `CILOGON_REFRESH_TOKEN` to generate new Fabric Identity Token and Fabric Refresh Token.<br>
For any subsequent use, we use Fabric Refresh Token. On every refresh, Fabric Refresh Token is changed and updated.<br>

NOTE: These steps are required for any experiments on Fabric Testbed.<br>

In [5]:
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token https://cilogon.org/oauth2/refreshToken/6daaa67c5e446bedf8b3af993f980e89/1604946297681
CILOGON_REFRESH_TOKEN environment variable: https://cilogon.org/oauth2/refreshToken/6b38eecaac7ba2d3378289b6eeab4b47/1604680448852


### Get new Fabric Identity Token and update Fabric Refresh Token

In [6]:
try:
    refresh_res = CredMgr.refresh_token(project_name='all', scope='all', refresh_token=fabric_refresh_token)
    print("New Tokens received: {}".format(json.dumps(refresh_res)))
    fabric_id_token=refresh_res['id_token']
    fabric_refresh_token=refresh_res['refresh_token']
    print()
    print("New Refresh Token: {}".format(fabric_refresh_token))
    print()
    print("Stored new Refresh Token")
    %store fabric_refresh_token
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

New Tokens received: {"id_token": "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJlbWFpbCI6Imt0aGFyZTEwQGVtYWlsLnVuYy5lZHUiLCJnaXZlbl9uYW1lIjoiS29tYWwiLCJmYW1pbHlfbmFtZSI6IlRoYXJlamEiLCJuYW1lIjoiS29tYWwgVGhhcmVqYSIsImlzcyI6Imh0dHBzOi8vY2lsb2dvbi5vcmciLCJzdWIiOiJodHRwOi8vY2lsb2dvbi5vcmcvc2VydmVyQS91c2Vycy8xMTkwNDEwMSIsImF1ZCI6ImNpbG9nb246L2NsaWVudF9pZC8xMjUzZGVmYzYwYTMyM2ZjYWEzYjQ0OTMyNjQ3NjA5OSIsInRva2VuX2lkIjoiaHR0cHM6Ly9jaWxvZ29uLm9yZy9vYXV0aDIvaWRUb2tlbi82NGU1NTQ5ODYxNjY1M2JhOWU3ZmZkMGVhZWJiNTkyOC8xNjA0OTQ2MTk1NzQwIiwiYXV0aF90aW1lIjoiMTYwNDk0NjE5NSIsImV4cCI6MTYwNDk0OTkxMywiaWF0IjoxNjA0OTQ2MzEzLCJyb2xlcyI6WyJDTzptZW1iZXJzOmFjdGl2ZSIsIkNPOkNPVTpKdXB5dGVyaHViOm1lbWJlcnM6YWN0aXZlIiwiQ086Q09VOnByb2plY3QtbGVhZHM6bWVtYmVyczphY3RpdmUiXSwic2NvcGUiOiJhbGwiLCJwcm9qZWN0IjoiYWxsIn0.ksfseAAu8XYVhmKIbbFtM9mX8PKZriwoUyhj9PsqvZxS-dczZFm_AC3TBxUxyTcjxMmmEO81SOUmh-8wL3sKW3e9aswS7DZxjI-Q13QLlYvxsXvYZfpF6PyitfW4VW6LA58DfiK0xr7VPU6Rw9hLPZD1M26kWzMAHltY9uyBGooWJtspgAUynrfGBVEqxXeQsM6ouxSwIXSYZPBFLYPTkEvyUOLoh-Qm